<a href="https://colab.research.google.com/github/EAName/yelp-reviews/blob/main/yelp_business_logistic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
import json
import os

import numpy as np
import pandas as pd
from tqdm import tqdm

In [3]:
df_business_linear =pd.read_csv('/content/gdrive/MyDrive/yelp_dataset/yelp_business_clean_version6.csv')


In [4]:
df_business_linear


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,Chicken Wings,Sports Bars,JapaneseTotal,HasTV_NA,RestaurantsPriceRange2_NA,RestaurantsReservations_NA,garage_NA,validated_NA,lot_NA,valet_NA
0,k0hlBqXX-Bt0vf1op7Jr1w,Tsevi's Pub And Grill,8025 Mackenzie Rd,Affton,MO,63123.0,38.565165,-90.321087,3.0,19,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,il_Ro8jwPlHresjw9EGmBg,Denny's,8901 US 31 S,Indianapolis,IN,46227.0,39.637133,-86.127217,2.5,28,...,0,0,0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
2,MUTTqe8uqyMdBl186RmNeA,Tuna Bar,205 Race St,Philadelphia,PA,19106.0,39.953949,-75.143226,4.0,245,...,0,0,1,0.0,0.0,0.0,1.0,1.0,0.0,0.0
3,ROeacJQwBeh05Rqg7F6TCg,BAP,1224 South St,Philadelphia,PA,19147.0,39.943223,-75.162568,4.5,205,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,9OG5YkX1g2GReZM0AskizA,Romano's Macaroni Grill,5505 S Virginia St,Reno,NV,89502.0,39.476117,-119.789339,2.5,339,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30925,gPr1io7ks0Eo3FDsnDTYfg,Tata Cafe,7201 Germantown Ave,Philadelphia,PA,19119.0,40.060414,-75.191084,4.0,21,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30926,wVxXRFf10zTTAs11nr4xeA,PrimoHoagies,6024 Ridge Ave,Philadelphia,PA,19128.0,40.032483,-75.214430,3.0,55,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30927,2MAQeAqmD8enCT2ZYqUgIQ,The Melting Pot - Nashville,"166 2nd Ave N, Ste A",Nashville,TN,37201.0,36.163875,-86.776311,4.0,204,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30928,w_4xUt-1AyY2ZwKtnjW0Xg,Bittercreek Alehouse,246 N 8th St,Boise,ID,83702.0,43.616590,-116.202383,4.5,998,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
df_business_linear.isna().any()


business_id                   False
name                          False
address                        True
city                          False
state                         False
                              ...  
RestaurantsReservations_NA    False
garage_NA                     False
validated_NA                  False
lot_NA                        False
valet_NA                      False
Length: 69, dtype: bool

In [6]:
df_business_linear.fillna({'address':0,
 'postal_code':0,
 'RestaurantsTakeOut':0,
 'HasTV':0,
 'BusinessAcceptsCreditCards':0,
 'BusinessParking':0,
 'RestaurantsPriceRange2':0,
 'RestaurantsReservations':0,
 },
 inplace=True)
df_business_linear.isna().any()

business_id                   False
name                          False
address                       False
city                          False
state                         False
                              ...  
RestaurantsReservations_NA    False
garage_NA                     False
validated_NA                  False
lot_NA                        False
valet_NA                      False
Length: 69, dtype: bool

In [7]:
df_business_linear

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,Chicken Wings,Sports Bars,JapaneseTotal,HasTV_NA,RestaurantsPriceRange2_NA,RestaurantsReservations_NA,garage_NA,validated_NA,lot_NA,valet_NA
0,k0hlBqXX-Bt0vf1op7Jr1w,Tsevi's Pub And Grill,8025 Mackenzie Rd,Affton,MO,63123.0,38.565165,-90.321087,3.0,19,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,il_Ro8jwPlHresjw9EGmBg,Denny's,8901 US 31 S,Indianapolis,IN,46227.0,39.637133,-86.127217,2.5,28,...,0,0,0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
2,MUTTqe8uqyMdBl186RmNeA,Tuna Bar,205 Race St,Philadelphia,PA,19106.0,39.953949,-75.143226,4.0,245,...,0,0,1,0.0,0.0,0.0,1.0,1.0,0.0,0.0
3,ROeacJQwBeh05Rqg7F6TCg,BAP,1224 South St,Philadelphia,PA,19147.0,39.943223,-75.162568,4.5,205,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,9OG5YkX1g2GReZM0AskizA,Romano's Macaroni Grill,5505 S Virginia St,Reno,NV,89502.0,39.476117,-119.789339,2.5,339,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30925,gPr1io7ks0Eo3FDsnDTYfg,Tata Cafe,7201 Germantown Ave,Philadelphia,PA,19119.0,40.060414,-75.191084,4.0,21,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30926,wVxXRFf10zTTAs11nr4xeA,PrimoHoagies,6024 Ridge Ave,Philadelphia,PA,19128.0,40.032483,-75.214430,3.0,55,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30927,2MAQeAqmD8enCT2ZYqUgIQ,The Melting Pot - Nashville,"166 2nd Ave N, Ste A",Nashville,TN,37201.0,36.163875,-86.776311,4.0,204,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30928,w_4xUt-1AyY2ZwKtnjW0Xg,Bittercreek Alehouse,246 N 8th St,Boise,ID,83702.0,43.616590,-116.202383,4.5,998,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression
import sklearn.metrics as metrics

In [9]:
X = df_business_linear.copy()
X = X.drop( ["stars","business_id","name","address","city","state","postal_code","latitude","longitude","attributes","categories","BusinessParking","market"], axis=1 )
Y = df_business_linear["stars"]

In [10]:
varNames = list( X.columns.values )
m = LinearRegression()
m.fit(X,Y)


coef_dict = {}
coef_dict["INTERCEPT"] = m.intercept_
for coef, feat in zip(m.coef_,varNames):
    coef_dict[feat] = coef

for i in coef_dict :
    print( i, " = ", coef_dict[i]  )


PRED = m.predict( X )
print(" Predicted Values ")
print( PRED )
print(" --------- ")


r2 = metrics.r2_score( Y, PRED )
print( "r2=", r2 )

mse = metrics.mean_squared_error(Y,PRED)
print( "MSE=", mse )

INTERCEPT  =  3.9378538790057926
review_count  =  0.0003916845828781254
is_open  =  0.13331329350008225
RestaurantsTakeOut  =  -0.027337747350925534
HasTV  =  -0.09548191664314803
BusinessAcceptsCreditCards  =  -0.22702532096378525
RestaurantsPriceRange2  =  -0.015536851556807674
RestaurantsReservations  =  0.038349408688232986
AgesAllowed_21plus  =  -0.38267021909704446
AgesAllowed_allages  =  -0.7155522724394714
Alcohol_None  =  0.06306787392416675
Alcohol_beer_and_wine  =  0.185708887742425
Alcohol_full_bar  =  -0.10554397432332918
BYOBCorkage_no  =  -0.12418612913200923
BYOBCorkage_yes_corkage  =  0.05491920582789786
BYOBCorkage_yes_free  =  -0.08880520420410651
NoiseLevel_average  =  -0.019729628779493275
NoiseLevel_loud  =  -0.189759423140979
NoiseLevel_quiet  =  0.05699430178420352
NoiseLevel_very_loud  =  -0.3996163745609313
RestaurantsAttire_casual  =  -0.23336960539761434
RestaurantsAttire_dressy  =  -0.032062761853418775
RestaurantsAttire_formal  =  -0.2971910982460051
Smoki

In [11]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
import sklearn.metrics as metrics

from sklearn import preprocessing

In [12]:
#le = preprocessing.LabelEncoder()

In [13]:
#le.fit(["paris", "paris", "tokyo", "amsterdam"])

In [16]:
Y = Y * 2

In [17]:
varNames = list( X.columns.values )
m = LogisticRegression()
m.fit(X,Y)




coef_dict = {}
coef_dict["INTERCEPT"] = m.intercept_[0]
for coef, feat in zip(m.coef_[0],varNames):
    coef_dict[feat] = coef

for i in coef_dict :
    print( i, " = ", coef_dict[i]  )



PRED_FLAG = m.predict( X )
print(" Predicted Flag Values ")
print( PRED_FLAG )
print(" --------- ")

PRED_PROB = m.predict_proba( X )
print(" Predicted Probabilities")
print( PRED_PROB )
print(" --------- ")

P1 = PRED_PROB[:,1]
print(" Predicted Probability of Class")
print( P1 )
print(" --------- ")



fpr_train, tpr_train, threshold = metrics.roc_auc_score( Y, P1, multi_class='ovr')
auc = metrics.auc(fpr_train, tpr_train)
print("AUC=",auc)

accuracy = metrics.accuracy_score( Y, PRED_FLAG )
print("ACCURACY=",accuracy)

INTERCEPT  =  -0.029130297525635167
review_count  =  -0.4761374266988408
is_open  =  -0.012023816415641951
RestaurantsTakeOut  =  -0.0274642661281927
HasTV  =  -0.019242533584344065
BusinessAcceptsCreditCards  =  -0.02687031525877314
RestaurantsPriceRange2  =  -0.03892970556772892
RestaurantsReservations  =  -0.008676774852636984
AgesAllowed_21plus  =  -8.334214482801133e-06
AgesAllowed_allages  =  -2.0137712551072288e-05
Alcohol_None  =  -0.011537068210580629
Alcohol_beer_and_wine  =  -0.0038968537394530145
Alcohol_full_bar  =  -0.0091844691683954
BYOBCorkage_no  =  -0.0004140222464574285
BYOBCorkage_yes_corkage  =  -4.0244616229966546e-05
BYOBCorkage_yes_free  =  -0.0002933978723133375
NoiseLevel_average  =  -0.010317179555423424
NoiseLevel_loud  =  -0.00061208991806647
NoiseLevel_quiet  =  -0.007448661368681282
NoiseLevel_very_loud  =  -0.000987697970000788
RestaurantsAttire_casual  =  -0.02442748417021868
RestaurantsAttire_dressy  =  -0.0002893452024885353
RestaurantsAttire_formal 

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


AxisError: ignored